In [3]:
import pandas as pd
import os

In [ ]:
notebook_dir = os.getcwd()

geocoded_csv_path = os.path.join(notebook_dir, '..', 'data', 'geocoded_addresses_final.csv')
geocoded = pd.read_csv(geocoded_csv_path)

checkpoint3_csv_path = os.path.join(notebook_dir, '..', 'data', 'checkpoints', 'checkpoint3_geocoded.csv')
checkpoint3 = pd.read_csv(checkpoint3_csv_path)

In [33]:
checkpoint3['address'] = checkpoint3['Location'].astype(str) + ', Lawrence, MA'

# merge with geocoded data
merged = checkpoint3.merge(
    geocoded.rename(columns={'latitude': 'geo_lat', 'longitude': 'geo_lon'}),
    on='address',
    how='left'
)

# replace latitude/longitude in checkpoint3 with values from geocoded where available
merged['Latitude'] = merged['geo_lat'].combine_first(merged['Latitude'])
merged['Longitude'] = merged['geo_lon'].combine_first(merged['Longitude'])

# drop helper columns
merged = merged.drop(columns=['address', 'geo_lat', 'geo_lon','cleaned_address'])

# save or use `merged` as the updated checkpoint3
checkpoint4 = merged

In [34]:
checkpoint4.head()

,Incident #,Date,Type,Location,Arrested,Location Prefix,Name,DOB,Charges,Latitude,Longitude
0,18015719.0,2018-04-01 00:02:35,ALARM/BURG,14 BROADWAY,No,FELO MOTORS,NaN,NaN,NaN,42.711509,-71.174821
1,18015720.0,2018-04-01 00:03:21,THREATS/PROG,131 SPRUCE ST,No,NaN,NaN,NaN,NaN,42.719151,-71.173651
2,18015721.0,2018-04-01 00:11:06,TOW OF M/V,481 S BROADWAY,No,TRESPASS,NaN,NaN,NaN,42.683771,-71.155965
3,18015722.0,2018-04-01 00:13:45,LOUD NOISE,550 BROADWAY,No,NaN,NaN,NaN,NaN,42.716258,-71.177648
4,18015724.0,2018-04-01 00:23:19,DOMESTIC/PAST,GARDEN ST & NEWBURY ST,No,NaN,NaN,NaN,NaN,42.709178,-71.157889


In [38]:
output_path = os.path.join(notebook_dir, '..', 'data', 'checkpoints', 'checkpoint4_geocoded.csv')
checkpoint4.to_csv(output_path, index=False)